# Visualisation des données

In [1]:
# Importation des librairies nécessaires
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sdkForMovies import MovieClient, MovieConfig
import time
import json
from collections import Counter, defaultdict
from pathlib import Path

d:\Dossier\Self_Training\Software\Certifications\Data_Science_Project_On_Udemy\movies-analytics\.venv\Lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [2]:
# Dossiers
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)